In [79]:
%reset

In [80]:
# -*- coding: utf-8 -*-
"""
Script for scraping METARS through site crawlling by means of automatized form submission 

"""
import pandas as pd
import numpy as np
import requests
import mechanicalsoup
from bs4 import BeautifulSoup
BeautifulSoup.DEFAULT_PARSER = 'lxml'
from datetime import datetime, timedelta


In [81]:
import metpy.io
import metpy.calc as mpcalc
from metpy.units import units


In [83]:

# input section
metars_lst = []
Icaos = ['EGLL', 'LFPG']
start_date = '01-2022' # FUTUREly INPUT
end_date = '02-2022'   # FUTUREly INPUT


# input pre-processing

parts = start_date.split('-')
start_month = int(parts[0])
start_year = int(parts[1])
parts = end_date.split('-')
end_month = int(parts[0])
end_year = int(parts[1])

import datescrape
dates = datescrape.generate_date_list(start_date, end_date) # list of dates in string format "MM-YYYY"

import calendict
month_length = calendict.generate_last_day_dictionary(start_year, end_year)



# URL of the page to crawl
url = "https://www.ogimet.com/metars.phtml.en"

METARS = pd.DataFrame(data=None, columns = ['METAR'])



for lugar in Icaos:

    for date in dates:

        # dates pre-processing

        parts = date.split('-')
        month = parts[0]
        year = parts[1]


        # Create a MechanicalSoup browser object
        browser = mechanicalsoup.StatefulBrowser()
        
        # Open the URL
        browser.open(url)
        
        # Get the form
        form = browser.select_form('form[action="display_metars2.php"]')
    
        # Fill out the form
        form['lugar'] = lugar
        form['tipo'] = 'SA'
        form['ord'] = 'REV'
        form['nil'] = 'NO'
        form['fmt'] = 'HTML'
        form['ano'] = year    # list dates
        form['mes'] = month     # list dates
        form['day'] = '01'      #fixo
        form['hora'] = '00'     #fixo
        form['anof'] = year   # list dates
        form['mesf'] = month     # list dates
        form['dayf'] = month_length[date]     # dictionary month_length
        form['horaf'] = '23'    #fixo


        # Submit the form
        browser.submit_selected()

        # Get the resulting HTML
        html_content = browser.get_current_page()

        # Extract the desired data
        data_table = html_content.find_all("pre")

        for metar in data_table:
            METARS_IT = metpy.io.parse_metar_to_dataframe(metar_text = metar.text, year = int(year), month = int(month))
            METARS = pd.concat([METARS, METARS_IT])
